# Building connection

In [1]:
import configparser
import psycopg2
import pandas as pd

config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

# Business Questions

In [2]:
# Define query executor
def exec_query(query, columns):
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns=columns)
    return df
    

#### What is the time period analysed?

In [3]:
%%time

query = """
select 
    min(start_time) as start_time, 
    max(start_time) as end_time 
from fact_songplay
"""

columns=['start_time','end_time']
df = exec_query(query, columns)

print(df.loc[:,:])

           start_time            end_time
0 2018-11-01 21:11:13 2018-11-30 17:31:24
CPU times: user 6.75 ms, sys: 1.9 ms, total: 8.65 ms
Wall time: 69.5 ms


#### Which locations have the most users using the app?

In [4]:
%%time

query = """
select 
    location,
    count(distinct user_id) as user_quantity
from fact_songplay
group by location
order by count(distinct user_id) desc
limit 15
"""

columns=['location', 'user_quantity']
df = exec_query(query, columns)

print(df.loc[:,:])

                                       location  user_quantity
0         New York-Newark-Jersey City, NY-NJ-PA              3
1          Houston-The Woodlands-Sugar Land, TX              3
2             Atlanta-Sandy Springs-Roswell, GA              3
3   Philadelphia-Camden-Wilmington, PA-NJ-DE-MD              2
4                                  Columbia, SC              2
5            San Jose-Sunnyvale-Santa Clara, CA              2
6                 San Antonio-New Braunfels, TX              2
7                   Detroit-Warren-Dearborn, MI              1
8                Pensacola-Ferry Pass-Brent, FL              1
9                                    Eugene, OR              1
10                               Santa Rosa, CA              1
11          Portland-Vancouver-Hillsboro, OR-WA              1
12              Dallas-Fort Worth-Arlington, TX              1
13                     New Orleans-Metairie, LA              1
14                                 Plymouth, IN        

#### What are the most played artists?

In [5]:
%%time

query = """
select 
    da.name as artist_name,
    count(fs.artist_id) as quantity
from fact_songplay as fs
join dim_artist as da
on da.artist_id = fs.artist_id
group by da.name
order by count(fs.artist_id) desc
limit 15
"""

columns=['artist_name', 'quantity']
df = exec_query(query, columns)

print(df.loc[:,:])

                                       artist_name  quantity
0                                    Dwight Yoakam        37
1                   Kid Cudi / Kanye West / Common        10
2                                         Kid Cudi        10
3                                       Ron Carter         9
4                                    Lonnie Gordon         9
5                                            B.o.B         8
6                                            Usher         6
7                   Usher featuring Jermaine Dupri         6
8                                             Muse         6
9   Richard Hawley And Death Ramps_ Arctic Monkeys         5
10                                  Arctic Monkeys         5
11                                      The Smiths         4
12                                       Metallica         4
13                                  Counting Crows         4
14                                 matchbox twenty         4
CPU times: user 3.24 ms,

#### What are the most played artists segmented by location?

In [6]:
%%time

query = """
select 
    da.name as artist_name,
    fs.location as location,
    count(fs.artist_id) as quantity
from fact_songplay as fs
join dim_artist as da
on da.artist_id = fs.artist_id
group by da.name, fs.location
order by count(fs.artist_id) desc
limit 15
"""

columns=['artist_name', 'location', 'quantity']
df = exec_query(query, columns)

print(df.loc[:,:])

                       artist_name                                 location  \
0                    Dwight Yoakam                 Lansing-East Lansing, MI   
1                    Dwight Yoakam        San Francisco-Oakland-Hayward, CA   
2                       Ron Carter        San Francisco-Oakland-Hayward, CA   
3                    Dwight Yoakam             Lake Havasu City-Kingman, AZ   
4                    Dwight Yoakam       Chicago-Naperville-Elgin, IL-IN-WI   
5                    Dwight Yoakam        Atlanta-Sandy Springs-Roswell, GA   
6                    Dwight Yoakam                         Marinette, WI-MI   
7                       The Smiths                 Lansing-East Lansing, MI   
8                            Usher      Tampa-St. Petersburg-Clearwater, FL   
9                   Arctic Monkeys        San Francisco-Oakland-Hayward, CA   
10  Usher featuring Jermaine Dupri              Portland-South Portland, ME   
11  Black Eyed Peas / Terry Dexter        San Franci

#### What are the most played songs?

In [7]:
%%time

query = """
select 
    ds.title as song_title,
    count(fs.song_id) as quantity
from fact_songplay as fs
join dim_song as ds
on ds.song_id = fs.song_id
group by ds.title
order by count(fs.song_id) desc
limit 15
"""

columns=['song_title', 'quantity']
df = exec_query(query, columns)

print(df.loc[:,:])

                                           song_title  quantity
0                                      You're The One        37
1                                 I CAN'T GET STARTED         9
2   Catch You Baby (Steve Pitron & Max Sanna Radio...         9
3   Nothin' On You [feat. Bruno Mars] (Album Version)         8
4                            Hey Daddy (Daddy's Home)         6
5                                        Make Her Say         5
6                                        Up Up & Away         5
7                                           Mr. Jones         4
8             Supermassive Black Hole (Album Version)         4
9                              Unwell (Album Version)         4
10                           Given Up (Album Version)         3
11                                      Fade To Black         3
12                                             Rianna         3
13                        From The Ritz To The Rubble         3
14                               Let's G

#### What are the most played songs segmented by location?

In [8]:
%%time

query = """
select 
    ds.title as song_title,
    fs.location as location,
    count(fs.song_id) as quantity
from fact_songplay as fs
join dim_song as ds
on ds.song_id = fs.song_id
group by ds.title, fs.location
order by count(fs.song_id) desc
limit 15
"""

columns=['artist_name', 'location', 'quantity']
df = exec_query(query, columns)

print(df.loc[:,:])

                                          artist_name  \
0                                      You're The One   
1                                      You're The One   
2                                 I CAN'T GET STARTED   
3                                      You're The One   
4                                      You're The One   
5                                      You're The One   
6                                Let's Get It Started   
7                                      You're The One   
8                                Girlfriend In A Coma   
9   Nothin' On You [feat. Bruno Mars] (Album Version)   
10                                     You're The One   
11                                     You're The One   
12                                   Caught Up In You   
13                                     You're The One   
14                           Hey Daddy (Daddy's Home)   

                                   location  quantity  
0                  Lansing-East

#### How many users do we have for the different existing levels?

In [9]:
%%time

query = """
select 
    level,
    count(user_id) as user_quantity
from dim_user
group by level
order by count(user_id) desc
limit 15
"""

columns=['level', 'user_quantity']
df = exec_query(query, columns)

print(df.loc[:,:])

  level  user_quantity
0  free             83
1  paid             22
CPU times: user 2.29 ms, sys: 725 µs, total: 3.02 ms
Wall time: 32.5 ms


#### What are the most popular user agents?

In [10]:
%%time

query = """
select 
    user_agent,
    count(songplay_id) as event_count
from fact_songplay
group by user_agent
order by count(songplay_id) desc
limit 15
"""

columns=['user_agent', 'event_count']
df = exec_query(query, columns)

print(df.loc[:,:])

                                           user_agent  event_count
0   "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...           51
1   Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...           42
2   "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...           34
3   "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...           28
4   "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...           22
5   Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...           22
6   "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...           17
7   "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...           16
8   Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...           13
9   "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....           11
10  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...           10
11  "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...           10
12  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_...            7
13  "Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...         

#### What are the most used user agents segmented by user level?

In [11]:
%%time

query = """
select 
    level,
    user_agent,
    count(songplay_id) as event_count
from fact_songplay
group by level, user_agent
order by count(songplay_id) desc
limit 15
"""

columns=['level', 'user_agent', 'event_count']
df = exec_query(query, columns)

print(df.loc[:,:])

   level                                         user_agent  event_count
0   paid  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...           47
1   paid  Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...           39
2   paid  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...           31
3   paid  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...           28
4   paid  Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...           20
5   paid  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...           15
6   paid  "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...           15
7   paid  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...           13
8   paid  Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...           10
9   paid  "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....           10
10  paid  "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...            9
11  paid  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_...            7
12  free  "Mozilla/5.0 (Windows NT 6.3; WOW64) Appl

# Close Connection

In [12]:
conn.close()